[align them in their goal to represent anatomy]

In [ ]:
#whats in a voxel

**whats in a streamline**

Before moving into a discussion of **how** our model of white matter anatomy is generated, we should consider an essential difference between the data representation of the T1 and the data reprsentation of white matter (i.e. tractography):

T1 images and other nifti data objects represent the brain with voxel-value pairings, such that each volume of represented space is associated with a quantative measure of that space.  As an arbitrary example, within some spatial frame of reference (i.e. scanner space, ACPC space, etc) a measured volume represented in the nifti image's data field at coordinate (128,120, 80) could have value of 125.32.  Relatedly, this is in much the same fashion that the 2D images that we discussed previously represent each area of depicted space with a value (or in the case of a standard color image, 3 values corresponding to RGB values).  Our most common method for representing white matter, which we refer to as "Tractography", DOES NOT operate in this fashion.  To see why lets refer back to the table that was provided just before we began our consideration of jpegs

|   | **Digital Photograph** | **Brain Image (T1)** | **Diffusion image** | **Tractography** |
| --- | --- | --- | --- | --- |
| _Object represented_ | visual scene | cranium / brain | cranium / brain | white matter of brain |
| _Source system_ | camera | MRI scanner | MRI scanner | Mathematical model  | 
| _Source phenomena_ | reflected light | water / magnetic properties | water movement | orientation interpolation |
| _Property of interest_ | topography | volumetric occupancy | tissue structure | putative axon collection traversal |
| _File extension_ | .jpg, .png ... | .nifti, nii.gz | [dwi] .nifti, nii.gz | .fg, .trk, .tck |
| _Metadata_ | exif | header | header | varies by format |
| _Data size_ | 100s kb - 1s MB | ~2.5 - 5 MB | 50 MB - 1.5 GB |500 MB - 10 GB |
| _Data dimensionality_ | &quot;2D&quot;(3 RGB layers) | 3D | 4D |1D nested? |
| _Data &quot;atoms&quot;_ | pixels | voxels | voxel-angle |vectors (streamlines) |
| _Data &quot;atom&quot; content_ | integer | float |float |ordered float sequence (nodes) |


assign a quantative value to each of its constituitive elements (streamlines).  Instead they merely have positional/spatial characteristics.

In a very general sense, the overarching goal of neuroimaging is to faithfully measure and represent the brain.  When we considered structural brain images (T1), we noted that this was not unlike the goal of standard photography, which attempts to capture the reflectance detail of a given scene.  In the case of neuroimaging though, we're doing this in 3D and grayscale.  This grayscale represents a quantative value associated with a particular volume of tissue.  While this is fine enough for capturing gross anatomical information (which would be useful in localizing strokes or brain lesions) it doesn't capture information that is specific to the essence of the gray and white matter.  To do this we need an imaging modality/capacity that asseses characteritics that are specific to that tissue:

For **gray matter** we need to assess *neural activity*, for **white matter** we need to assess *axonal connectivity*.  These needs dictate the reprsentational elements we use.  With *activity*, voxels are still sufficient, though we consider them stretched across time.  With *connectivity* though, voxels are insufficient, and so we move to streamlines.

**Lets consider what a streamline is for a moment**

In [1]:
import nibabel as nib
import numpy as np
import os

#Path to the repo for this project
repoPath='/Users/plab/Documents/ipynb/'

# load a tractography file with a single streamline in it
streamsObjIN=nib.streamlines.load(os.path.join(repoPath,'singleStream.tck'))

# determine the number of streamlines
print(list(np.shape(streamsObjIN.tractogram.streamlines)))

[1, 142, 3]


Above, we have loaded a .tck file ("singleStream.tck") and then printed out the dimensions of the object storing streamlines.  Here we see that it is 1 by 142 by 3.  This indicates that there is 1 streamline with 142 nodes across three dimensions (X, Y, and Z).  This is a fairly unique tractography file, in that it only has one streamline in it.  Typically, a tractography file will contain many thousands of streamlines.  In the case of a tractography file that is representing an entire brain's white matter, it will likely have millions of streamlines.  For now though, we'll focus on just one streamline to get a sense of what a streamline is.

This streamline (like others) is an ordered sequence of nodes (points in three dimensional space) representing a path through the white matter of the brain.  **This should not be interpreted as representing a single axon**.  Rather, given the coarseness of our diffusion measure, and the necessarily inferential methods involved with generating these streamlines (we use our diffusion data to estimate where we should put the streamline), it would be better to interpret this as **"our best guess as to where there appears to be a collection of axons continuously oriented in the same direction"**.  The specifics of how these guesses are generated can be discussed elsewhere (within the context of tractography generation algorithms), but for now this description is sufficient to begin considering how streamlines and collections of streamlines represent the brain's white matter.  

Lets plot this streamline below to see what one of these white matter representations looks like.

In [ ]:
from niwidgets import StreamlineWidget
from niwidgets.exampledata import streamlines

sw = StreamlineWidget(filename=streamlines)
style = {'axes': {'color': 'red',
                  'label': {'color': 'white'},
                  'ticklabel': {'color': 'white'},
                  'visible': False},
         'background-color': 'white',
         'box': {'visible': False}}
sw.plot(display_fraction=0.5, width=500, height=500, style=style, percentile=80)



In the case of **gray matter**, it is the activity of the neurons that is of particular salience, however a static image cannot capture this.  As such, researchers often turn to functional MRI scanning to assess the activities of cortical tissue over a given period of time. In this way, the data structure for fMRI is extended over a 4th dimension, time.  This allows researchers to investigate how blood oxygen levels (**BOLD**, a proxy measure for neural activity) evolve over time, and associate these changes with experimental manipulations. However, the **white mater** does not contain neuron bodies, and thus typically does not exhibit the kind of biophysical processes that are measured by fMRI. Moreover, what we are intreseted in, relative to **white matter** is not it's *activity* but rather, its *connectivity*.  Just as fMRI does not measure brain/neuronal activity directly, the imaging modality used in the study of **white mater**, diffusion MRI, doesn't measure the characteristic of interest (axonal structure/connectivity) directly.  Instead, diffusion MRI measures the propensity of water to move in a specific direction.  Because **myelin** (the lipid-dense neural structure component which gives white matter it's lighter apperance) restricts water from escaping out of an axon, water thus tends to move in a direction that is perpindicular to the axon (in other words, along the length of the axon). 

Each of these more specialized approaches to investigating brain tissue requires a specialized processing method. In both cases, the processing is specific to a given voxel, as is the characteristic that is to be inferred from looking at these measures.  For fMRI, the temporal structure of the **BOLD** signal must be inferred from the changes observed between sampling time points (i.e. slices in the fMRI data object's 4th dimension).  For dMRI the axonal structure must be inferred from a multitude of measurements (taken at various angular orientations) of water's diffusion propensity.  Interestingly though, in order to explore connective characteristics of the white matter, this derived model of diffusion propensity must be used as the foundation for the generation of another model which attempts to encapsulate the gross axonal structure of the white matter.



Given that we are attempting to use these brain images for research, it seems that we would want these to be of the highest resolution possble so that we can see the brain's constiuent components in in fine detail.  Given our previous look at nifti data stuctures, we know that the resolution is actually fairly coarse (~ 1mm^3).  But just for the sake of argument, lets think about how many voxels would it take to represent the brain on a cellular level.  

**What sort of information would we need to make an informed estimate about this?**

Well, first we would need an estimate of how many neurons are in the cerebral cortex. Herculano-Houzel notes in a 2009 review (https://doi.org/10.3389/neuro.09.031.2009) that a good estimate of this number is actually a recent development, and that most numbers cited were unsubstantiated.  The number provided by Herculano-Houzel is **16 billion** neurons in the cerebral cortex (though **69 billion** appear to be located in the cerebellum), and approximately **85 billion non-neuronal cells**



In [12]:
#preloading and processing
import nibabel as nib
import numpy as np
t1Path='/Users/plab/Documents/JupyerData/proj-5c50b6f12daf2e0032f880eb/sub-100206/dt-neuro-anat-t1w.tag-acpc_aligned.tag-brain_extracted.id-5c57072befbc2800526291bb/t1.nii.gz'
img = nib.load(t1Path)
print('Voxel dimensions for T1 (in mm)')
voxelDims=img.header.get_zooms()
print(voxelDims)
print('')

volData = img.get_fdata()
unwrappedData=np.ndarray.flatten(volData)

def largeVal(n):
    return n>0

result=map(largeVal,unwrappedData)
largeBool=list(result)
largeNum=sum(largeBool)
print('Number of non zero (data containing) voxels')
print(largeNum)



Voxel dimensions for T1 (in mm)
(1.25, 1.25, 1.25)

Number of data occupied voxels
972417


In [17]:
#lets assume all cells are layed out in a latice matrix (i.e. they are all orthogonal to each other and they can thus be represented as a matrix.)
print('Estimated number neurons')
#citation
neuronNum=100000000000
print(neuronNum)
print('')

print('Estimated number of glia')
gliaNum=neuronNum*10
print(gliaNum)
print('')




#neuronBodySize=25*10^-6
#gliaBodySize=15*10^-6

#neuronVolume=[neuronBodySize^3]*neuronNum
#gliaVolume=[gliaBodySize^3]*gliaNum

totalCellNum=neuronNum+gliaNum

print('Number of cells per voxel (estimate)')
cellPerVox=totalCellNum/largeNum
print(cellPerVox)


print('Number of cells per cubic mm (estimate)')
cellPerMM=cellPerVox/(voxelDims[0]*voxelDims[1]*voxelDims[2])
print(cellPerMM)
print('')

axonNum=neuronNum

print('Current data storage usage for T1')
import os
statinfo = os.stat(t1Path)
T1bytes=statinfo.st_size
print(f'{T1bytes} bytes' )
T1gigabytes=T1bytes/1073741824
print(f'{T1gigabytes} gigabytes' )
print('')

print('Storage for a ')
percellT1bytes=cellPerVox*largeNum*8
print(f'{percellT1bytes} bytes' )
percellT1gigabytes=percellT1bytes/1073741824
print(f'{percellT1gigabytes} gigabytes' )





Estimated number neurons
100000000000

Estimated number of glia
1000000000000

Number of cells per voxel (estimate)
1131201.9431992653
Number of cells per cubic mm (estimate)
579175.3949180238

Current data storage usage for T1
3593891 bytes
0.0033470718190073967 gigabytes

Current data storage usage for T1
8800000000000.0 bytes
8195.638656616211 gigabytes
